In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from difflib import SequenceMatcher
import os
# !pip install -q tqdm
from tqdm.notebook import tqdm

In [2]:
driver_options = Options()
driver_options.add_argument("accept-language=en-US")

s=Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s,options=driver_options)
driver.maximize_window()

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [C:\Users\emman\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache


In [3]:
INITIAL_WEBSITE = "https://torrent9.to/"
data = {}

In [4]:
# driver.find_element(By.CLASS_NAME, 'nav navbar-nav')#.click()
# element = driver.find_element(by=By.CLASS_NAME, value="header-top")

In [5]:
driver.get(INITIAL_WEBSITE)

In [6]:
[cb.click() for cb in driver.find_elements(by=By.CSS_SELECTOR, value="dic.checkbox")]
[cb.click() for cb in driver.find_elements(by=By.CSS_SELECTOR, value="div.captcha")]

[]

In [7]:
header = driver.find_elements(by=By.CSS_SELECTOR, value="#bs-example-navbar-collapse-1 > ul a")

categories = header[:-1]
categories = {i.text: i.get_attribute('href') for i in categories}


for (category, link) in (pbar :=tqdm(categories.items())):
    # print(f"category {category} -- link: {link}")
    pbar.set_description("Processing category : %s" % category)
    next_page_link = link
    
    actual_page = 1
    is_first_page = True
    is_last_page_scraped = False
    list_of_tables = []
    
    while not is_last_page_scraped:
        driver.get(next_page_link)
        # first page processing
        if is_first_page:
            # print(f"first page processing")
            try:
                second_page_link = driver.find_element(by=By.CSS_SELECTOR, value="#pagination-mian > nav > ul > li:nth-child(2) > a").get_attribute('href')
                last_page_btn = driver.find_element(by=By.CSS_SELECTOR, value="#pagination-mian > nav > ul > li:last-child > a")
                last_page_link, n_pages = last_page_btn.get_attribute('href'), int(last_page_btn.text)
                match = SequenceMatcher(None, second_page_link, last_page_link).find_longest_match()
                base_link = second_page_link[match.a:match.a + match.size]
            except:
                n_pages = 1
            pbar_ = tqdm(total=n_pages, leave=False) # Progress bar of this category
            is_first_page = False # change the boolean to no more do the processing for the next pages
        # scrape content
        pbar_.set_description("Scraping page %s" % actual_page)
        # print("Scraping in processing")
        table = driver.find_element(by=By.CSS_SELECTOR, value="body > section > div > div > div > div > div.row.cus-row > div.col-sm-12.cus-col.content-left-col > div.left-tab-section > div.table-responsive ")
        df = pd.read_html(table.get_attribute('innerHTML'))[0]
        a_tags = table.find_elements(by=By.CSS_SELECTOR, value="div.table-responsive > table > tbody > tr > td > a")
        links = [a.get_attribute('href') for a in a_tags]
        df = pd.read_html(table.get_attribute('innerHTML'))[0]
        df["link"] = links
        # df.insert(-1, "link", links )
        list_of_tables.append(df)


        # list_of_tables = []

        # checking
        if actual_page == n_pages :
            is_last_page_scraped = True
            # print("Last page scraped")
        else:
            # prepare to move to the next page
            actual_page += 1
            next_page_link = base_link+ f"{actual_page}"
            # print(f"Let's go to the page '{actual_page}' : {next_page_link}")
        pbar_.update(1)

        # print('')
        # break
    table_stack = pd.concat(list_of_tables, axis=0)
    n_items = table_stack.shape[0]
    table_stack.to_csv(os.path.join("data", f"{category}.csv"), index=False)
    del table_stack
    del list_of_tables
    pbar_.close()
    # break
    
    

0it [00:00, ?it/s]

In [7]:
# driver.get(INITIAL_WEBSITE)
# header = driver.find_elements(by=By.CSS_SELECTOR, value="#bs-example-navbar-collapse-1 > ul a")

# for category in header:
#     print(f"category {category.text} -- link: {category.get_attribute('href')}")

#     # go to a page from navbar
#     driver.get(category.get_attribute('href'))
#     last_page = False
#     list_of_tables = []

#     while not last_page:
#         table = driver.find_element(by=By.CSS_SELECTOR, value="body > section > div > div > div > div > div.row.cus-row > div.col-sm-12.cus-col.content-left-col > div.left-tab-section > div.table-responsive ")
#         df = pd.read_html(table.get_attribute('innerHTML'))[0]
#         a_tags = table.find_elements(by=By.CSS_SELECTOR, value="div.table-responsive > table > tbody > tr > td > a")
#         links = [a.get_attribute('href') for a in a_tags]
#         df = pd.read_html(table.get_attribute('innerHTML'))[0]
#         df["link"] = links
#         # df.insert(-1, "link", links )
#         list_of_tables.append(df)

#         page_bar = driver.find_element(by=By.CSS_SELECTOR, value="#pagination-mian > nav > ul")
#         pages = page_bar.find_elements(by=By.CSS_SELECTOR, value="li > a")
#         try:
#             next_page = page_bar.find_element(by=By.CSS_SELECTOR, value="li.active + li").find_element(by=By.CSS_SELECTOR, value="a")
#             next_page.click()
#         except:
#             # print("Last page")
#             last_page = True

#         # break

#     table_stack = pd.concat(list_of_tables, axis=0)
#     n_items = table_stack.shape[0]
#     # n_pages = int(page_bar.find_element(by=By.CSS_SELECTOR, value="li.active").find_element(by=By.CSS_SELECTOR, value="a").text)
#     data[category] = {'n_items': n_items, 'df': table_stack, }
#     # break

In [8]:
table_stack.head()

NameError: name 'table_stack' is not defined

In [24]:
table_stack.shape

(450, 6)